In [1]:
import pyarrow.fs
from ray.data import ActorPoolStrategy
import sycamore
from sycamore.functions.tokenizer import HuggingFaceTokenizer
from sycamore.llms import OpenAIModels, OpenAI
from sycamore.transforms import COALESCE_WHITESPACE
from sycamore.transforms.merge_elements import MarkedMerger
from sycamore.transforms.partition import SycamorePartitioner
from sycamore.transforms.extract_schema import OpenAISchemaExtractor, OpenAIPropertyExtractor
from sycamore.transforms.embed import SentenceTransformerEmbedder

In [2]:
from sycamore.data.document import Document
from dateutil import parser
def convert_timestamp(doc: Document) -> Document:
    if "dateAndTime" not in doc.properties['entity'] and "dateTime" not in doc.properties['entity']:
        return doc
    raw_date: str = doc.properties['entity'].get('dateAndTime') or doc.properties['entity'].get('dateTime')
    raw_date = raw_date.replace("Local", "")
    parsed_date = parser.parse(raw_date, fuzzy=True)
    extracted_date = parsed_date.date()
    doc.properties['entity']['day'] = extracted_date.day
    doc.properties['entity']['month'] = extracted_date.month
    doc.properties['entity']['year'] = extracted_date.year
    if parsed_date.utcoffset():
        doc.properties['entity']['dateTime'] = parsed_date.isoformat()
    else:
        doc.properties['entity']['dateTime'] = parsed_date.isoformat() + "Z"
    
    return doc

In [3]:
paths = ["s3://aryn-public/ntsb/"]
fsys = pyarrow.fs.S3FileSystem(region="us-east-1", anonymous=True)

llm = OpenAI(OpenAIModels.GPT_3_5_TURBO.value)
tokenizer = HuggingFaceTokenizer("thenlper/gte-small")

ctx = sycamore.init()

ds = (
    ctx.read.binary(paths, binary_format="pdf", filesystem=fsys)
    # Parition with the sycamore partitioner, pulling out tables and images. ActorPoolStrategy(size=3) works best on my particular hardware
    # but your mileage may vary depending on your RAM.
    .partition(partitioner=SycamorePartitioner(extract_table_structure=True, extract_images=True, use_cache=False), compute=ActorPoolStrategy(size=3))
    # Get rid of spurious whitespace charaters
    .regex_replace(COALESCE_WHITESPACE)
    # Automatically determine a schema of additional metadata to extract from documents
    .extract_batch_schema(schema_extractor=OpenAISchemaExtractor("FlightAccidentReport", llm=llm, num_of_elements=35))
    # Extract the metadata specified by that schema
    .extract_properties(property_extractor=OpenAIPropertyExtractor(llm=llm, num_of_elements=35))
    # Merge elements into larger chunks
    .mark_bbox_preset(tokenizer=tokenizer)
    .merge(merger=MarkedMerger())
    # Convert extracted timestamps to better-structured form using the function above
    .map(convert_timestamp)
    # Copy document properties to each document's sub-elements
    .spread_properties(["path", "entity"])
    # Split elements that are too big to embed
    .split_elements(tokenizer=tokenizer, max_tokens=512)
    # Convert all Elements to Documents
    .explode()
    # Generate a series of hashes to represent each document. For use with near-duplicate detection
    .sketch()
    # Embed each document
    .embed(embedder=SentenceTransformerEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2", batch_size=100))
)

In [4]:
# Write to a persistent duckdb database
# - into a specific database (as specified by url) 
# - into a specific table (as specified by table name)
persistent_db = "demo.db"
persistent_table = "demo_table"
ds.write.duckdb(
    db_url=persistent_db,
    table_name=persistent_table,
    dimensions=384
)

2024-07-05 12:06:31,061	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-05 12:06:31,062	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-05 12:06:31,066	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-05 12:06:31,070	INFO streaming_executor.py:112 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-05_12-06-27_345614_98706/logs/ray-data
2024-07-05 12:06:31,070	INFO streaming_executor.py:113 -- Execution plan o

- ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap) 1:   0%|          | 0/…

- MapBatches(regex_replace)->MapBatches(BaseMapTransformCustom__Extract) 2:   0%|          | 0/65 [00:00<?, ?i…

- MapBatches(extract_properties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_d…

Running 0:   0%|          | 0/65 [00:00<?, ?it/s]

...

(MapWorker(ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap)) pid=98733) The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
(MapWorker(ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap)) pid=98734) The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


<IPython.core.display.HTML object>erties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatches(mark_break_page)->MapBatches(mark_break_by_column)->MapBatches(mark_break_by_tokens)->MapBatches(merge_elements)->MapBatches(convert_timestamp)->MapBatches(spread_properties)->MapBatches(split_doc)->MapBatches(explode)->MapBatches(sketcher)->MapBatches(SentenceTransformerEmbedder)->MapBatches(BaseMapTransformCallable__duckdb_write_documents)) pid=98736)  [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
<IPython.core.display.HTML object>erties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatches(mark_break_page)->MapBatches(mark_break_by_column)->MapBatches(mark_break_by_tokens)->MapBatc

(MapWorker(MapBatches(extract_properties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatches(mark_break_page)->MapBatches(mark_break_by_column)->MapBatches(mark_break_by_tokens)->MapBatches(merge_elements)->MapBatches(convert_timestamp)->MapBatches(spread_properties)->MapBatches(split_doc)->MapBatches(explode)->MapBatches(sketcher)->MapBatches(SentenceTransformerEmbedder)->MapBatches(BaseMapTransformCallable__duckdb_write_documents)) pid=98736) /Users/karansampath/Library/Caches/pypoetry/virtualenvs/sycamore-ai--zTjaFUY-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
(MapWorker(MapBatches(extract_properties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatc

<IPython.core.display.HTML object>erties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatches(mark_break_page)->MapBatches(mark_break_by_column)->MapBatches(mark_break_by_tokens)->MapBatches(merge_elements)->MapBatches(convert_timestamp)->MapBatches(spread_properties)->MapBatches(split_doc)->MapBatches(explode)->MapBatches(sketcher)->MapBatches(SentenceTransformerEmbedder)->MapBatches(BaseMapTransformCallable__duckdb_write_documents)) pid=98736)  [repeated 60x across cluster]
<IPython.core.display.HTML object>erties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatches(mark_break_page)->MapBatches(mark_break_by_column)->MapBatches(mark_break_by_tokens)->MapBatches(merge_elements)->MapBatches(convert_timestamp)->MapBatches(spread_properties)->MapBatches(split_doc)->MapBatches(explode)->MapBatches(sketcher)->MapBatches(SentenceTransformerEmbedder)->MapBatches(BaseMapTran

(MapWorker(MapBatches(extract_properties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatches(mark_break_page)->MapBatches(mark_break_by_column)->MapBatches(mark_break_by_tokens)->MapBatches(merge_elements)->MapBatches(convert_timestamp)->MapBatches(spread_properties)->MapBatches(split_doc)->MapBatches(explode)->MapBatches(sketcher)->MapBatches(SentenceTransformerEmbedder)->MapBatches(BaseMapTransformCallable__duckdb_write_documents)) pid=98736) /Users/karansampath/miniforge3/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(MapWorker(MapBatches(extract_properties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_drop_header_and_footer)->MapBatches(mark_break_page)->MapBatches(mark_break_by_column)->MapBatches(mark_break_by_tokens)->MapBatches(merge_elements)->MapBatches(convert_timestamp)->Map

In [6]:
# We connect to the DuckDB to perform operations
import duckdb
data_conn = duckdb.connect("demo.db")

In [7]:
# DuckDB is not production ready yet for efficient Vector Similarity Search on persistent databases (https://duckdb.org/docs/extensions/vss.html), 
# so we load our data into an in-memory database for this demo. In future versions, once DuckDB productionizes this feature, there will be no more need for this code cell

# Load from disk into Pandas Dataframe as an intermediate step
df = data_conn.execute("SELECT * FROM demo_table").fetchdf()
df.dropna(subset=["embeddings"],inplace=True)

# Now, we load the data into an in-memory database. Notice that we specify beforehand since we need the embedding column (must be specified as a FLOAT[N] where N is specified) 
# for Vector Similarity Search using HNSW to work
schema = {
            "doc_id": "VARCHAR",
            "embeddings": "FLOAT[384]",
            "properties": "MAP(VARCHAR, VARCHAR)",
            "text_representation": "VARCHAR",
            "bbox": "DOUBLE[]",
            "shingles": "BIGINT[]",
            "type": "VARCHAR",
        }
in_memory_db = duckdb.connect(":default:")
in_memory_db.install_extension("vss")
in_memory_db.load_extension("vss")
in_memory_db.execute(f"""CREATE TABLE in_memory_table (doc_id {schema.get('doc_id')},
                      embeddings {schema.get('embeddings')}, properties {schema.get('properties')}, 
                      text_representation {schema.get('text_representation')}, bbox {schema.get('bbox')}, 
                      shingles {schema.get('shingles')}, type {schema.get('type')})"""
    )
in_memory_db.execute("""INSERT INTO in_memory_table SELECT * FROM df; 
                      CREATE INDEX in_memory_table_index ON in_memory_table USING HNSW(embeddings)""")

In [8]:
# For queries, let's define an embedding function for the question that helps us easily compare the two vectors and generate optimal results // Run NN search
from sentence_transformers import SentenceTransformer
minilm = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

class Embedder():
    def __init__(self, llm):
        self.llm = llm
    
    def embed_query(self, question):
        v = self.llm.encode(question).tolist()
        return v

embedder = Embedder(minilm)

/Users/karansampath/Library/Caches/pypoetry/virtualenvs/sycamore-ai--zTjaFUY-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
q = "What automobile type is the most accident prone?"
in_memory_db.sql(f"SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_distance(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10;")
# Pretty print tables
# text representation --> print one cell out (of the top result). 

┌──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────────────┐
│        doc_id        │ text_representation  │                               properties                               │
│       varchar        │       varchar        │                         map(varchar, varchar)                          │
├──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────────────────┤
│ 9061a466-abae-4889…  │ The National Trans…  │ {score=0.8915606737136841, page_numbers=[1], page_number=1, path=s3:…  │
│ a382f8ad-70a1-4e89…  │ The National Trans…  │ {score=0.8774372339248657, page_numbers=[1], page_number=1, path=s3:…  │
│ 3be945d2-eb3c-412a…  │ The National Trans…  │ {score=0.7743222713470459, page_numbers=[1], page_number=1, path=s3:…  │
│ a6bd8f8c-759f-42ae…  │ The National Trans…  │ {score=0.7248827219009399, page_numbers=[1], page_number=1, path=s3:…  │
│ 4d5d7f1e-b035-406c…  │ The Nat

In [10]:
# Let us now see the performance of the ANN HNSW search using DuckDB. We use Array Euclidean Distance as the nearest neighbor lookup

from tabulate import tabulate

q = "What automobile type is the most accident prone?"
result = in_memory_db.sql(f"SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_distance(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10;")

# Convert the result to a list of lists
table_data = result.fetchall()

# Get column names
headers = [desc[0] for desc in result.description]

# Pretty print the table
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+--------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
first_two = table_data[2:4]

print("Text representation of the first two values (not including headers):")
for i, row in enumerate(first_two, 1):
    print(f"\n{i}. doc_id: {row[0]}")
    print(f"   Text representation:\n   {row[1]}")
    print("-" * 50)

Text representation of the first two values (not including headers):

1. doc_id: 3be945d2-eb3c-412a-8beb-620d0d82b16f
   Text representation:
   The National Transportation Safety Board determines the probable cause(s) of this accident to be:
The partial loss of engine power for reasons that could not be determined.
Page 1 of 6

--------------------------------------------------

2. doc_id: a6bd8f8c-759f-42ae-9861-d31d24c05c5b
   Text representation:
   The National Transportation Safety Board determines the probable cause(s) of this accident to be:
The pilot’s failure to maintain airspeed during a drag maneuver, which resulted in an overrun and subsequent impact with terrain.
Page 1 of 5

--------------------------------------------------


In [12]:
# We can also see the performance of the DuckDB query and check the index is being used by using the in-built visualization below 
q = "What automobile type is the most accident prone?"
explain_result = in_memory_db.sql(f"EXPLAIN SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_distance(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10")

# Convert the result to a list of lists
explain_data = explain_result.fetchall()

# Get column names
headers = [desc[0] for desc in explain_result.description]

# Pretty print the EXPLAIN output
print("Expanded EXPLAIN output:")
print(tabulate(explain_data, headers=headers, tablefmt="grid"))

Expanded EXPLAIN output:
+---------------+------------------------------------------------------------+
| explain_key   | explain_value                                              |
+===============+============================================================+
| physical_plan | ┌───────────────────────────┐                              |
|               | │         PROJECTION        │                              |
|               | │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                              |
|               | │             #0            │                              |
|               | │             #1            │                              |
|               | │             #2            │                              |
|               | └─────────────┬─────────────┘                              |
|               | ┌─────────────┴─────────────┐                              |
|               | │         PROJECTION        │                              |
|               | │   ─ ─ ─

In [13]:
# Finally we can build an index with other distance metrics, the default is Euclidean Distance ('l2sq'), with Cosine similarity ('cosine', 'array_cosine_simarlity') 
# and Inner Product ('ip', 'array_inner_product') also supported
in_memory_db.execute("CREATE INDEX in_memory_ip_index ON in_memory_table USING HNSW(embeddings) WITH (metric = 'ip')")
in_memory_db.sql(f"SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_inner_product(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10;")

┌──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────────────┐
│        doc_id        │ text_representation  │                               properties                               │
│       varchar        │       varchar        │                         map(varchar, varchar)                          │
├──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────────────────┤
│ 9061a466-abae-4889…  │ The National Trans…  │ {score=0.8915606737136841, page_numbers=[1], page_number=1, path=s3:…  │
│ a382f8ad-70a1-4e89…  │ The National Trans…  │ {score=0.8774372339248657, page_numbers=[1], page_number=1, path=s3:…  │
│ 3be945d2-eb3c-412a…  │ The National Trans…  │ {score=0.7743222713470459, page_numbers=[1], page_number=1, path=s3:…  │
│ a6bd8f8c-759f-42ae…  │ The National Trans…  │ {score=0.7248827219009399, page_numbers=[1], page_number=1, path=s3:…  │
│ 4d5d7f1e-b035-406c…  │ The Nat

In [14]:
# We can now perform RAG on this table using Langchain. To make it work, we must add a metadata column that contains a 'source' value which we do
# as a placeholder below
in_memory_db.sql("ALTER TABLE in_memory_table ADD COLUMN IF NOT EXISTS metadata VARCHAR DEFAULT '{\"source\": \"default\"}'")


In [15]:
# We now initialize a vector store on DuckDB using the LangChain integration. We perform similarity search using the integration here
from langchain_community.vectorstores.duckdb import DuckDB
vector_store = DuckDB(connection=in_memory_db, embedding=embedder, id_key= "doc_id", text_key="text_representation", vector_key="embeddings", table_name="in_memory_table")
result = vector_store.similarity_search('traffic')
print(result)

[Document(page_content='Passenger Information\n', metadata={'source': 'default', '_similarity_score': 0.3923537060189796}), Document(page_content='Passenger Information\n', metadata={'source': 'default', '_similarity_score': 0.3923537060189796}), Document(page_content='Passenger Information\n', metadata={'source': 'default', '_similarity_score': 0.3923537060189796}), Document(page_content='Passenger Information\n', metadata={'source': 'default', '_similarity_score': 0.3923537060189796})]


In [16]:
from langchain_openai import ChatOpenAI  
from langchain.chains import RetrievalQAWithSourcesChain  
import os
# Finally, we initialize a RAG agent and ask a model question of the data
llm = ChatOpenAI(  
    openai_api_key=os.environ.get("OPENAI_API_KEY"),  
    model_name='gpt-3.5-turbo',  
    temperature=0.8  
)  
qa = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vector_store.as_retriever() , verbose=True
)  
qa.invoke({"question": "How many accidents happened?"})



> Entering new RetrievalQAWithSourcesChain chain...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



> Finished chain.


{'question': 'How many accidents happened?',
 'answer': 'There were a total of 7 accidents that happened.\n',
 'sources': 'default'}

In [17]:
# OPTIONAL: If you would like to remove your database after running the code above, please run this code cell
try:
    os.unlink(persistent_db)
except Exception as e:
    print(f"Error deleting {persistent_db}: {e}")